In [5]:
# !pip install Pillow
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install tensorflow
# !pip install keras-tuner

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import PIL
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import scipy


%matplotlib inline

# Testing Different Batch Size and Learning Rate

In [2]:
base_dir = 'data'
mask_dir = os.path.join(base_dir, 'with_mask')
wo_mask_dir = os.path.join(base_dir, 'without_mask')

In [3]:
mask_files = list(os.walk(mask_dir))[0][2]
print(f'number of mask pictures: {len(mask_files)}')
wo_mask_files = list(os.walk(wo_mask_dir))[0][2]
print(f'number of mask pictures: {len(wo_mask_files)}')

number of mask pictures: 3725
number of mask pictures: 3828


In [10]:
def test_minibatch_learning_rate(mini_batch_size, learning_rate,epochs=5):
    model = tf.keras.models.Sequential([
        # Note the input shape is the desired size of the image 224x224 with 3 bytes color
        # This is the first convolution
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The fourth convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The fifth convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('with mask') and 1 for the other ('without mask')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=learning_rate),
                metrics=['accuracy'])
    batch_size = mini_batch_size
    # All images will be rescaled by 1./255
    train_datagen = ImageDataGenerator(rescale=1./255, 
                                    rotation_range = 40,
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True,
                                    brightness_range=[0.4,1.5],
                                    validation_split=0.2) # 20% split and it is not random

    val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # created another generator for validation without augmentation

    # Flow training images in batches of 128 using train_datagen generator
    train_generator = train_datagen.flow_from_directory(
            base_dir,  # This is the source directory for training images
            target_size=(224, 224),  # All images will be resized to 224x224
            batch_size=batch_size,
            # Since we use binary_crossentropy loss, we need binary labels
            class_mode='binary',
            shuffle=True,
            subset='training') #set as training data

    validation_generator = val_datagen.flow_from_directory(
            base_dir,  # This is the source directory for training images
            target_size=(224, 224),  # All images will be resized to 224x224
            batch_size=batch_size,
            # Since we use binary_crossentropy loss, we need binary labels
            class_mode='binary',
            shuffle=False,
            subset='validation') #set as validation data
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,  
        validation_data = validation_generator,
        validation_steps = validation_generator.samples // batch_size,
        epochs=epochs,
        initial_epoch=0,
        verbose=1)
    return history

In [11]:
test_minibatch_learning_rate(128, 0.001, 5)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/5
38/47 [=======================>......] - ETA: 1:00 - loss: 0.6935 - accuracy: 0.5872

c:\Users\Bao\Desktop\comp-vision-py\project_env\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


47/47 [==============================] - 328s 7s/step - loss: 0.6725 - accuracy: 0.6166 - val_loss: 0.5080 - val_accuracy: 0.9126
Epoch 2/5
47/47 [==============================] - 285s 6s/step - loss: 0.4941 - accuracy: 0.7772 - val_loss: 0.3645 - val_accuracy: 0.9276
Epoch 3/5
47/47 [==============================] - 280s 6s/step - loss: 0.4488 - accuracy: 0.7981 - val_loss: 0.2736 - val_accuracy: 0.9439
Epoch 4/5
47/47 [==============================] - 304s 6s/step - loss: 0.4058 - accuracy: 0.8221 - val_loss: 0.2034 - val_accuracy: 0.9290
Epoch 5/5
47/47 [==============================] - 299s 6s/step - loss: 0.3792 - accuracy: 0.8399 - val_loss: 0.1919 - val_accuracy: 0.9332


In [12]:
test_minibatch_learning_rate(128, 0.01, 5)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/5
47/47 [==============================] - 311s 7s/step - loss: 171.1599 - accuracy: 0.6385 - val_loss: 0.2754 - val_accuracy: 0.9112
Epoch 2/5
47/47 [==============================] - 317s 7s/step - loss: 0.5668 - accuracy: 0.7374 - val_loss: 0.9365 - val_accuracy: 0.6697
Epoch 3/5
47/47 [==============================] - 291s 6s/step - loss: 0.7195 - accuracy: 0.5055 - val_loss: 0.6929 - val_accuracy: 0.5291
Epoch 4/5
47/47 [==============================] - 285s 6s/step - loss: 0.6935 - accuracy: 0.5033 - val_loss: 0.6959 - val_accuracy: 0.4709
Epoch 5/5
47/47 [==============================] - 284s 6s/step - loss: 0.6935 - accuracy: 0.4920 - val_loss: 0.6924 - val_accuracy: 0.5291


In [13]:
test_minibatch_learning_rate(128, 0.0001, 5)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/5
47/47 [==============================] - 306s 6s/step - loss: 0.6218 - accuracy: 0.6600 - val_loss: 0.4692 - val_accuracy: 0.8224
Epoch 2/5
47/47 [==============================] - 31726s 690s/step - loss: 0.5065 - accuracy: 0.7686 - val_loss: 0.3485 - val_accuracy: 0.8757
Epoch 3/5
47/47 [==============================] - 273s 6s/step - loss: 0.4735 - accuracy: 0.7897 - val_loss: 0.2985 - val_accuracy: 0.8949
Epoch 4/5
47/47 [==============================] - 277s 6s/step - loss: 0.4492 - accuracy: 0.8024 - val_loss: 0.2807 - val_accuracy: 0.9048
Epoch 5/5
47/47 [==============================] - 268s 6s/step - loss: 0.4347 - accuracy: 0.8107 - val_loss: 0.2603 - val_accuracy: 0.8970


In [14]:
test_minibatch_learning_rate(64, 0.0005, 5)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/5
94/94 [==============================] - 274s 3s/step - loss: 0.5875 - accuracy: 0.7102 - val_loss: 0.3073 - val_accuracy: 0.8981
Epoch 2/5
94/94 [==============================] - 277s 3s/step - loss: 0.4512 - accuracy: 0.7931 - val_loss: 0.2451 - val_accuracy: 0.9185
Epoch 3/5
94/94 [==============================] - 326s 3s/step - loss: 0.4011 - accuracy: 0.8262 - val_loss: 0.2167 - val_accuracy: 0.9382
Epoch 4/5
94/94 [==============================] - 340s 4s/step - loss: 0.3516 - accuracy: 0.8468 - val_loss: 0.1710 - val_accuracy: 0.9484
Epoch 5/5
94/94 [==============================] - 2335s 25s/step - loss: 0.3243 - accuracy: 0.8630 - val_loss: 0.1558 - val_accuracy: 0.9497


In [15]:
test_minibatch_learning_rate(64, 0.001, 5)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/5
94/94 [==============================] - 150s 2s/step - loss: 0.5924 - accuracy: 0.7063 - val_loss: 0.3452 - val_accuracy: 0.9212
Epoch 2/5
94/94 [==============================] - 152s 2s/step - loss: 0.4219 - accuracy: 0.8090 - val_loss: 0.2487 - val_accuracy: 0.9117
Epoch 3/5
94/94 [==============================] - 152s 2s/step - loss: 0.3615 - accuracy: 0.8480 - val_loss: 0.1956 - val_accuracy: 0.9212
Epoch 4/5
94/94 [==============================] - 155s 2s/step - loss: 0.3224 - accuracy: 0.8660 - val_loss: 0.1511 - val_accuracy: 0.9416
Epoch 5/5
94/94 [==============================] - 149s 2s/step - loss: 0.3154 - accuracy: 0.8710 - val_loss: 0.1868 - val_accuracy: 0.9463


In [16]:
test_minibatch_learning_rate(256, 0.001, 5)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/5
23/23 [==============================] - 142s 6s/step - loss: 0.8104 - accuracy: 0.5789 - val_loss: 0.4890 - val_accuracy: 0.8719
Epoch 2/5
23/23 [==============================] - 145s 6s/step - loss: 0.5077 - accuracy: 0.7664 - val_loss: 0.5301 - val_accuracy: 0.7094
Epoch 3/5
23/23 [==============================] - 145s 6s/step - loss: 0.5116 - accuracy: 0.7655 - val_loss: 0.3008 - val_accuracy: 0.9133
Epoch 4/5
23/23 [==============================] - 144s 6s/step - loss: 0.4566 - accuracy: 0.7970 - val_loss: 0.2320 - val_accuracy: 0.9195
Epoch 5/5
23/23 [==============================] - 140s 6s/step - loss: 0.4328 - accuracy: 0.8025 - val_loss: 0.2090 - val_accuracy: 0.9344


In [17]:
test_minibatch_learning_rate(256, 0.002, 5)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/5
23/23 [==============================] - 145s 6s/step - loss: 1.5519 - accuracy: 0.5531 - val_loss: 0.4914 - val_accuracy: 0.8578
Epoch 2/5
23/23 [==============================] - 144s 6s/step - loss: 0.6385 - accuracy: 0.6964 - val_loss: 0.6571 - val_accuracy: 0.5898
Epoch 3/5
23/23 [==============================] - 152s 7s/step - loss: 0.6680 - accuracy: 0.6789 - val_loss: 0.6889 - val_accuracy: 0.5570
Epoch 4/5
23/23 [==============================] - 156s 7s/step - loss: 0.5986 - accuracy: 0.7208 - val_loss: 0.6710 - val_accuracy: 0.5547
Epoch 5/5
23/23 [==============================] - 153s 7s/step - loss: 0.5662 - accuracy: 0.7301 - val_loss: 0.4302 - val_accuracy: 0.8727


In [18]:
test_minibatch_learning_rate(256, 0.00141, 5)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/5
23/23 [==============================] - 151s 6s/step - loss: 1.1114 - accuracy: 0.5440 - val_loss: 0.7957 - val_accuracy: 0.5820
Epoch 2/5
23/23 [==============================] - 143s 6s/step - loss: 0.6424 - accuracy: 0.6473 - val_loss: 0.6992 - val_accuracy: 0.5453
Epoch 3/5
23/23 [==============================] - 144s 6s/step - loss: 0.6365 - accuracy: 0.6627 - val_loss: 0.6413 - val_accuracy: 0.6352
Epoch 4/5
23/23 [==============================] - 147s 6s/step - loss: 0.6305 - accuracy: 0.7031 - val_loss: 0.2809 - val_accuracy: 0.9102
Epoch 5/5
23/23 [==============================] - 146s 6s/step - loss: 0.5245 - accuracy: 0.7719 - val_loss: 0.2779 - val_accuracy: 0.9312


Mini batch size of 64 seems to perform the best

Trying out different learning rate for a batch size of 64 for 20 epochs

In [19]:
test_minibatch_learning_rate(64, 0.0005, 20)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 148s 2s/step - loss: 0.5372 - accuracy: 0.7428 - val_loss: 0.2426 - val_accuracy: 0.9321
Epoch 2/20
94/94 [==============================] - 168s 2s/step - loss: 0.4005 - accuracy: 0.8254 - val_loss: 0.2071 - val_accuracy: 0.9212
Epoch 3/20
94/94 [==============================] - 171s 2s/step - loss: 0.3569 - accuracy: 0.8476 - val_loss: 0.1662 - val_accuracy: 0.9416
Epoch 4/20
94/94 [==============================] - 149s 2s/step - loss: 0.3205 - accuracy: 0.8629 - val_loss: 0.1718 - val_accuracy: 0.9450
Epoch 5/20
94/94 [==============================] - 155s 2s/step - loss: 0.3074 - accuracy: 0.8690 - val_loss: 0.1361 - val_accuracy: 0.9511
Epoch 6/20
94/94 [==============================] - 156s 2s/step - loss: 0.2870 - accuracy: 0.8796 - val_loss: 0.1616 - val_accuracy: 0.9524
Epoch 7/20
94/94 [==============================] - 159s 2s/step - los

In [20]:
test_minibatch_learning_rate(64, 0.001, 20)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 184s 2s/step - loss: 0.6269 - accuracy: 0.6670 - val_loss: 0.3599 - val_accuracy: 0.8974
Epoch 2/20
94/94 [==============================] - 187s 2s/step - loss: 0.4657 - accuracy: 0.7881 - val_loss: 0.2203 - val_accuracy: 0.9253
Epoch 3/20
94/94 [==============================] - 187s 2s/step - loss: 0.3956 - accuracy: 0.8307 - val_loss: 0.1685 - val_accuracy: 0.9450
Epoch 4/20
94/94 [==============================] - 184s 2s/step - loss: 0.3539 - accuracy: 0.8570 - val_loss: 0.1756 - val_accuracy: 0.9382
Epoch 5/20
94/94 [==============================] - 166s 2s/step - loss: 0.3300 - accuracy: 0.8694 - val_loss: 0.1749 - val_accuracy: 0.9348
Epoch 6/20
94/94 [==============================] - 167s 2s/step - loss: 0.2999 - accuracy: 0.8814 - val_loss: 0.1425 - val_accuracy: 0.9538
Epoch 7/20
94/94 [==============================] - 172s 2s/step - los

both learning rate seem to perform similarly but the lower learning rate seem to perform better at the start so we will be going with a learning rate of 0.0005 and a mini batch size of 64